In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy import io, signal, stats
import sys, os
sys.path.append('../')

import pandas as pd
from sklearn.decomposition import PCA
import fooof
from fooof import FOOOFGroup
from seaborn import despine
import matplotlib.pyplot as plt

from galaxybrain.data_utils import load_mouse_data, return_pops
from galaxybrain import ramsey
from neurodsp.spectral import compute_spectrum

import warnings
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

from galaxybrain import analysis_pipe

In [7]:
datafolder = '../data/spikes/' #where the mouse data is
all_mice = []
for i_m in range(3):
    print('Mouse ' + (str(i_m+1)))
    all_mice.append([])
    df_spk, df_info = load_mouse_data(datafolder, i_m, return_type='binned', bin_width=1)
    region_indices = {}
    for region in df_info.region.unique():
        region_indices.update({region:np.where(df_info['region'] == str(region))[0]})

    spk_list, region_labels = return_pops(df_spk, df_info)
    print(list(zip(region_labels, [s.shape[1] for s in spk_list])), 'Total:',sum([s.shape[1] for s in spk_list]))

    su_start_ind = len(region_labels)+1

    all_mice[i_m].append(df_spk[df_spk.columns[su_start_ind:]])
    all_mice[i_m].append(region_indices)

krebs = all_mice[0]; robbins = all_mice[1]; waksman = all_mice[2]

mice_regions = {'krebs': [krebs, [('all', 1462), ('CP', 176), ('HPF', 265), ('LS', 122), ('MB', 127), ('TH', 227), ('V1', 334)]],
            'robbins': [robbins, [('all', 2688), ('FrMoCtx', 647), ('HPF', 333), ('LS', 133), ('RSP', 112), ('SomMoCtx', 220), ('TH', 638), ('V1', 251), ('V2', 124)]],
            'waksman': [waksman, [('all', 2296), ('CP', 134), ('HPF', 155), ('TH', 1878)]] }

Mouse 1
Grabbing Spikes...
Binning Spikes...
[('CP', 176), ('FrMoCtx', 78), ('HPF', 265), ('LS', 122), ('MB', 127), ('SC', 42), ('SomMoCtx', 91), ('TH', 227), ('V1', 334)] Total: 1462
Mouse 2
Grabbing Spikes...
Binning Spikes...
[('FrCtx', 31), ('FrMoCtx', 647), ('HPF', 333), ('LS', 133), ('MB', 40), ('RSP', 112), ('SC', 98), ('SSCtx', 50), ('SomMoCtx', 220), ('TH', 638), ('UNK', 11), ('V1', 251), ('V2', 124)] Total: 2688
Mouse 3
Grabbing Spikes...
Binning Spikes...
[('CP', 134), ('FrMoCtx', 86), ('HPF', 155), ('LH', 4), ('LS', 4), ('RSP', 10), ('SomMoCtx', 10), ('TH', 1878), ('UNK', 15)] Total: 2296


In [ ]:
burn_in = 20
output_dir = '../data/experiments/expTEST'
ramsey_params = {'n_iters' : 3, 'n_pc' : 0.8, 'f_range' : [0,0.4]}
num_trials = 2
mouse_in = ['krebs']
shuffle = ('space',4)
"""
ANALYSIS AS SCRIPT TO DEBUG
"""
#analysis + saving data 
for mouse_key in mouse_in:
    mouse = mice_regions[mouse_key][0] #this is the data
    for region in mice_regions[mouse_key][1]:

        region_name = region[0]; region_count = region[1]
        print(region_name)
        os.makedirs(f'{output_dir}/{mouse_key}/{region_name}')

        if region_name == 'all':
            mouse_raster = mouse[0].iloc[burn_in:-burn_in]
        else:
            mouse_raster = mouse[0][mouse[1][region_name]].iloc[burn_in:-burn_in] #mouse[1] are indices

        subsetsizes = np.linspace(30,region_count,16, dtype=int)

        if shuffle:
            for s in range(shuffle[1]):
                curr_raster = analysis_pipe.shuffle_data(mouse_raster, shuffle[0]) 
                curr_output = {'eigs':[],'pows':[],'pca_m':[],'s_er':[],'ft_m':[],'t_er':[],'psn_r':[], 'spn_r':[], 'psn_p':[], 'spn_p':[]}
                curr_output = []
                for n in range(num_trials):
                    eigs, pows, pca_m, s_er, ft_m, t_er, psn_r, spn_r, psn_p, spn_p = ramsey.ramsey(curr_raster, subsetsizes, **ramsey_params)
                    curr_output.append([eigs, pows, pca_m, s_er, ft_m, t_er, psn_r, spn_r, psn_p, spn_p])
                # AVG ACROSS TRIALS HERE
                curr_output = np.array(curr_output)
                np.savez(f'{output_dir}/{mouse_key}/{region_name}/ramsey_{s+1}', eigs=np.array([curr_output[:,0][i] for i in range(num_trials)]).mean(0), # this properly takes the mean over trials
                                                                                 pows=np.array([curr_output[:,1][i] for i in range(num_trials)]).mean(0), # ^
                                                                                 pca_m=curr_output[:,2].mean(0), space_er=curr_output[:,3].mean(0), 
                                                                                 ft_m=curr_output[:,4].mean(0), time_er=curr_output[:,5].mean(0), 
                                                                                 pearson_r=curr_output[:,6].mean(0), spearman_rho=curr_output[:,7].mean(0), 
                                                                                 pearson_p=curr_output[:,8].mean(0), spearman_p=curr_output[:,9].mean(0))
        else:
            for i in range(num_trials):
                eigs, pows, pca_m, s_er, ft_m, t_er, psn_r, spn_r, psn_p, spn_p = ramsey.ramsey(mouse_raster, subsetsizes, **ramsey_params)
                np.savez(f'{output_dir}/{mouse_key}/{region_name}/ramsey_{i+1}', eigs=eigs, pows=pows, pca_m=pca_m, space_er=s_er, ft_m=ft_m, time_er=t_er, pearson_r=psn_r, spearman_rho=spn_r, pearson_p=psn_p, spearman_p=spn_p)

## Misc

In [7]:
def load_data(files = None, show_dir = False):
    """ Load data from your data folder
    If show_dir is True, displays a list of files in your directory.  Omits the extension.
    If show_dir is False, assumes input is a string of format 'file1, file2 ...'
    Only really works for .csv for now
    """
    l = os.listdir('../data')
    l.remove('.DS_Store')
    if show_dir == True:
        for i, x in enumerate(l):
            x = x[:x.index('.')]
            l[i] = x
        print('Files in ../data: ', l)
        files = input("Enter file name(s). Separate with commas for multiple files.  Enter 'all' (w/o quotes)  to load all \n")
        print('Uploading...')
        if files == 'all':
            for z in l:
                globals()[z] = pd.read_csv("../data/"+ z +".csv", index_col = 0)
            print('Done.')

        else:
            files = files.split(',')
            files = [i.translate({ord(i):None for i in ' '}) for i in files] # takes out spaces

            for z in files:
                globals()[z] = pd.read_csv("../data/"+ z +".csv", index_col = 0)
            print('Done.')

    if show_dir == False:
        print('Uploading...')
        if files == 'all':
            for z in l:
                globals()[z[:z.index('.')]] = pd.read_csv("../data/"+ z, index_col = 0)
            print('Done.')
        else:
            files = files.split(',')
            files = [i.translate({ord(i):None for i in ' '}) for i in files] # takes out spaces
            for z in files:
                globals()[z] = pd.read_csv("../data/"+ z +".csv", index_col = 0)
            print('Done.')
